<a href="https://colab.research.google.com/github/DataEtnos/Apache_Spark/blob/main/SparkUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install flask-ngrok
!pip install pyngrok

In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
      .config('spark.ui.port', '4050')
      .appName("SparkUI Introdução")
      .getOrCreate()
)

In [ ]:
!ngrok config add-authtoken '2pp0qW25AGqGYK6xftvt4Q4XXLv_JsvPyKtvYupn7z65UHQ5'

In [ ]:
# {
#     "id_transacao": 1000,
#     "valor": "58931.97",
#     "remetente": {"nome": "Jonathan Gonsalves", "banco": "BTG", "tipo": "PF"},
#     "destinatario": {"nome": "Emanuella Moura", "banco": "Itau", "tipo": "PJ"},
#     "transaction_date": "2021-06-02",
#     "chave_pix": "aleatoria",
#     "fraude": "1"
# }

from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType()),
])


schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('transaction_date', TimestampType()),
    StructField('chave_pix', StringType()),
    StructField('fraude', IntegerType())
])


caminho_json = './pix_transactions.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd"
)

In [ ]:
df.show()

In [ ]:
df.select('destinatario.nome').show()

In [ ]:
df.write.mode('overwrite').partitionBy('chave_pix').parquet('outpute/pix')

In [ ]:
!nrgrok authtoken 2pp0qW25AGqGYK6xftvt4Q4XXLv_JsvPyKtvYupn7z65UHQ5
from flask import Flask


app = Flask(__name__)

@app.route('/')
def home():
  return"hello , this is your flask app running on google colab with ngrok"

from pyngrok import ngrok
public_url = ngrok.connect(4050)
print('link:', public_url)
app.run()


/bin/bash: line 1: nrgrok: command not found


link: NgrokTunnel: "https://e872-34-74-134-245.ngrok-free.app" -> "http://localhost:4050"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


# SparkSQL